In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"C:\Users\matte\OneDrive\Desktop\GitHub\data\causal\sb_exp_every.csv")

df.head()

,d,delivery_time,delivery_time_1,delivery_time_0,tau
0,1,2.84,2.84,5.84,-3.0
1,0,4.49,1.49,6.49,-5.0
2,0,7.27,2.27,8.27,-6.0
3,1,5.27,2.27,8.27,-6.0
4,1,5.59,4.59,10.59,-6.0


In [2]:
df_lags = df.assign(**{
             f"d_l{l}" : df["d"].shift(l) for l in range(7)
         })

df_lags[[f"d_l{l}" for l in range(7)]].head(10)

,d_l0,d_l1,d_l2,d_l3,d_l4,d_l5,d_l6
0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1.0,NaN,NaN,NaN,NaN,NaN
2,0,0.0,1.0,NaN,NaN,NaN,NaN
3,1,0.0,0.0,1.0,NaN,NaN,NaN
4,1,1.0,0.0,0.0,1.0,NaN,NaN
5,0,1.0,1.0,0.0,0.0,1.0,NaN
6,1,0.0,1.0,1.0,0.0,0.0,1.0
7,1,1.0,0.0,1.0,1.0,0.0,0.0
8,0,1.0,1.0,0.0,1.0,1.0,0.0
9,0,0.0,1.0,1.0,0.0,1.0,1.0


In [3]:
import statsmodels.formula.api as smf

model = smf.ols("delivery_time ~" + "+".join([f"d_l{l}"
                                                       for l in range(7)]),
                         data=df_lags).fit()

model.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,9.3270,0.461,20.246,0.000,8.414,10.240
d_l0,-2.9645,0.335,-8.843,0.000,-3.629,-2.300
d_l1,-1.8861,0.339,-5.560,0.000,-2.559,-1.213
d_l2,-1.0013,0.340,-2.943,0.004,-1.676,-0.327
d_l3,0.2594,0.341,0.762,0.448,-0.416,0.935
d_l4,0.1431,0.340,0.421,0.675,-0.531,0.817
d_l5,0.1388,0.340,0.408,0.684,-0.536,0.813
d_l6,0.5588,0.336,1.662,0.099,-0.108,1.225


In [4]:
tau_m_hat = model.params[1:4].sum() 
se_tau_m_hat = np.sqrt((model.bse[1:4]**2).sum()) 
print("tau_m:", tau_m_hat)
print("95% CI:", [tau_m_hat -1.96*se_tau_m_hat,
                           tau_m_hat +1.96*se_tau_m_hat])

tau_m: -5.8518568954422925
95% CI: [-7.000105171362163, -4.703608619522422]


In [5]:
rad_points_3 = np.array([True, False, False]*(2))
rad_points_3

array([ True, False, False,  True, False, False])

In [6]:
rad_points_3.cumsum()

array([1, 1, 1, 2, 2, 2])

In [7]:
from numpy.lib.stride_tricks import sliding_window_view

m = 2
sliding_window_view(rad_points_3.cumsum(), window_shape=m+1)

array([[1, 1, 1],
       [1, 1, 2],
       [1, 2, 2],
       [2, 2, 2]])

In [8]:
np.diff(sliding_window_view(rad_points_3.cumsum(), 3), axis=1)

array([[0, 0],
       [0, 1],
       [1, 0],
       [0, 0]])

In [10]:
n_rand_windows = np.concatenate([
             [np.nan]*m,
             np.diff(sliding_window_view(rad_points_3.cumsum(), 3),
                     axis=1).sum(axis=1)+1
         ])

n_rand_windows

array([nan, nan,  1.,  2.,  2.,  1.])

In [11]:
p=0.5
p**n_rand_windows

array([ nan,  nan, 0.5 , 0.25, 0.25, 0.5 ])

In [13]:
def compute_p(rand_points, m, p=0.5):
             n_windows_last_m = np.concatenate([
                 [np.nan]*m,
                 np.diff(sliding_window_view(rand_points.cumsum(), m+1),
                         axis=1).sum(axis=1)+1
             ])
             return p**n_windows_last_m

compute_p(np.ones(6)==1, 2, 0.5)

array([  nan,   nan, 0.125, 0.125, 0.125, 0.125])